In [1]:
## import some modules between which also our hammer_fit

import numpy as np
import pyhf
import matplotlib.pyplot as plt
import json
from redist import modifier
from redist import hammer_fit

In [2]:
# you can define your fitter in a json file and it will contain all the informations about:
#   - WCs to pass to Hammer
#   - FF parametrization in use
#   - the location of the file containing the hammer histograms

reader = hammer_fit.Reader("./B02DstTauNu_config.json")

# createFitter produces a fitter object able handle the hammer histograms and produce the templates
fitter = reader.createFitter()

# store in your fitter your data (this is not necessary as you can always fit external data)
# the parameters are defined in the json file and here we produce a SM like toy we aim to fit
params_toy = {"SM" : 1., "Re_S_qLlL" : 0., "Im_S_qLlL" : 0., "Re_S_qRlL" : 0., "Im_S_qRlL" : 0.,  "Re_V_qLlL" : 0., "Im_V_qLlL" : 0. , "Re_V_qRlL" : 0., "Im_V_qRlL" : 0., "Re_T_qLlL" : 0., "Im_T_qLlL" : 0., "delta_RhoSq" : 0., "delta_cSt" : 0., "delta_chi21" : 0., "delta_chi2p" : 0., "delta_chi3p" : 0., "delta_eta1" : 0., "delta_etap" : 0., "delta_phi1p" : 0., "delta_beta21" : 0., "delta_beta3p" : 0.,"lumi" : 1.}
toy_data = fitter._template_list[0].generate_toy(**params_toy) + fitter._template_list[1].generate_toy(**params_toy)

fitter.upload_data(toy_data)

In [3]:
# definition of the alternative model (the nul model is stored in the fitter object)
SM_dist = fitter.nul_pdf()

def NP_dist(**kwargs):
    params_NP = {"SM" : kwargs["SM"], "Re_S_qLlL" : kwargs["Re_S_qLlL"], "Im_S_qLlL" : kwargs["Im_S_qLlL"], "Re_S_qRlL" : kwargs["Re_S_qRlL"], "Im_S_qRlL" : kwargs["Im_S_qRlL"],  "Re_V_qLlL" : kwargs["Re_V_qLlL"], "Im_V_qLlL" : kwargs["Im_V_qLlL"], "Re_V_qRlL" : kwargs["Re_V_qRlL"], "Im_V_qRlL" : kwargs["Im_V_qRlL"], "Re_T_qLlL" : kwargs["Re_T_qLlL"], "Im_T_qLlL" : kwargs["Im_T_qLlL"],"delta_RhoSq" : kwargs["FFs[8]"], "delta_cSt" : kwargs["FFs[0]"], "delta_chi21" : kwargs["FFs[6]"], "delta_chi2p" : kwargs["FFs[2]"], "delta_chi3p" : kwargs["FFs[9]"], "delta_eta1" : kwargs["FFs[7]"], "delta_etap" : kwargs["FFs[3]"], "delta_phi1p" : kwargs["FFs[1]"], "delta_beta21" : kwargs["FFs[5]"], "delta_beta3p" : kwargs["FFs[4]"],"lumi" : 1.}
    res = np.zeros(fitter._template_list[0]._nobs)
    for temp in fitter._template_list:
        res += temp.generate_template(**params_NP)
    return res

In [4]:
# loading of the model created in B02DstTauNu_model.ipynb
model, alt_yields = hammer_fit.load_hammer('B02DstTauNu_model.json', NP_dist, SM_dist, return_data=True)
data = alt_yields + model.config.auxdata

In [5]:
# defining the starting values of the d.o.f.
init = model.config.suggested_init()
init[0] = 1. #SM
init[1] = 0. #ReSLL
init[2] = 0. #ImSLL
init[3] = 0. #ReSRL
init[4] = 0. #ImSRL
init[5] = 0. #ReVLL
init[6] = 0. #ImVLL
init[7] = 0. #ReVRL
init[8] = 0. #ImVRL
init[9] = 0. #ReTLL
init[10] = 0. #ImTLL

init[11] = 0. #FF1
init[12] = 0. #FF2
init[13] = 0. #FF3
init[14] = 0. #FF4
init[15] = 0. #FF5
init[16] = 0. #FF6
init[17] = 0. #FF7
init[18] = 0. #FF8
init[19] = 0. #FF9
init[20] = 0. #FF10

init[21] = 1. #mu nominal

In [6]:
# fixing the values of some of the d.o.f. (here we want to fit the Tensor WC)
fixed = model.config.suggested_fixed()
#fixed[0] = True #SM
fixed[1] = True #ReSLL
fixed[2] = True #ImSLL
fixed[3] = True #ReSRL
fixed[4] = True #ImSRL
fixed[5] = True #ReVLL
fixed[6] = True #ImVLL
fixed[7] = True #ReVRL
fixed[8] = True #ImVRL
#fixed[9] = True #ReTLL
#fixed[10] = True #ImTLL

#fixed[11] = True #FF1
#fixed[12] = True #FF2
#fixed[13] = True #FF3
#fixed[14] = True #FF4
#fixed[15] = True #FF5
#fixed[16] = True #FF6
#fixed[17] = True #FF7
#fixed[18] = True #FF8
#fixed[19] = True #FF9
#fixed[20] = True #FF10
fixed[21] = True # nomi mu

In [7]:
# pyhf fit and print of the best values

pyhf.set_backend("numpy", pyhf.optimize.scipy_optimizer())
best_fit = pyhf.infer.mle.fit(data, model, fixed_params=fixed, init_pars=init)
pyhf.set_backend("numpy", pyhf.optimize.minuit_optimizer(verbose=True, strategy=1))
best_fit, twice_nll, OptimizeResult = pyhf.infer.mle.fit(
    data,
    model,
    init_pars=best_fit.tolist(),
    fixed_params=fixed,
    return_uncertainties=True,
    return_fitted_val=True,
    return_result_obj=True
    )

best_fit_dict = modifier.par_dict(model, best_fit)
for i in range(len(best_fit)):
    print(f"{model.main_model.modifiers_appliers['custom'].param_viewer.allpar_viewer.names[i]} = {best_fit[i][0]} +/- {best_fit[i][1]}")

/home/mcolonna/.conda/envs/env_2024_11_25/lib/python3.10/site-packages/redist/hammer_fit.py:58: RuntimeWarning: invalid value encountered in divide
  weights = np.array(alt_binned) / np.array(self.null_binned)


SM = 0.9966800963810507 +/- 0.0034006668159239917
Re_S_qLlL = 0.0 +/- 0.0
Im_S_qLlL = 0.0 +/- 0.0
Re_S_qRlL = 0.0 +/- 0.0
Im_S_qRlL = 0.0 +/- 0.0
Re_V_qLlL = 0.0 +/- 0.0
Im_V_qLlL = 0.0 +/- 0.0
Re_V_qRlL = 0.0 +/- 0.0
Im_V_qRlL = 0.0 +/- 0.0
Re_T_qLlL = -0.0028327367995586833 +/- 0.003317981734413544
Im_T_qLlL = -0.009100852661866995 +/- 0.03134921867718032
FFs_decorrelated[0] = 0.2738007336711573 +/- 0.9727869835540026
FFs_decorrelated[1] = -0.07557665148936826 +/- 0.9861742036568851
FFs_decorrelated[2] = 0.17158081958766294 +/- 0.9216841813484493
FFs_decorrelated[3] = 0.10367804280146958 +/- 0.9723653558669244
FFs_decorrelated[4] = 0.10922334093868764 +/- 0.9817316331025636
FFs_decorrelated[5] = 0.018727624395168035 +/- 0.9643325212672463
FFs_decorrelated[6] = 0.24029162237180512 +/- 0.7991486965763861
FFs_decorrelated[7] = 0.08546897077036839 +/- 0.9391713457053478
FFs_decorrelated[8] = -0.08756687699131639 +/- 0.6282628298977726
FFs_decorrelated[9] = 0.22345646684750053 +/- 0.94544